# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 12:15PM,243210,20,ALUR253669881,Alumier Labs Inc.,Released
1,Aug 2 2022 12:15PM,243210,20,ALUR344016207,Alumier Labs Inc.,Released
2,Aug 2 2022 12:15PM,243210,20,ALUR596135469,Alumier Labs Inc.,Released
3,Aug 2 2022 12:15PM,243210,20,ALUR919818782,Alumier Labs Inc.,Released
4,Aug 2 2022 12:15PM,243210,20,ALU0007423,Alumier Labs Inc.,Released
5,Aug 2 2022 12:15PM,243210,20,ALUR045551506,Alumier Labs Inc.,Released
6,Aug 2 2022 12:15PM,243210,20,ALUR582781079,Alumier Labs Inc.,Released
7,Aug 2 2022 12:15PM,243210,20,ALUR876042246,Alumier Labs Inc.,Released
8,Aug 2 2022 12:15PM,243210,20,ALUR371287725,Alumier Labs Inc.,Released
9,Aug 2 2022 12:15PM,243210,20,ALUR816002890,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,243267,Executing,19
45,243267,Released,14
46,243268,Released,5
47,243272,Released,2
48,243273,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243266,NaN,15.0,4.0
243267,NaN,19.0,14.0
243268,NaN,NaN,5.0
243272,NaN,NaN,2.0
243273,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243170,34.0,11.0,0.0
243173,0.0,0.0,16.0
243174,0.0,14.0,23.0
243185,0.0,0.0,1.0
243204,3.0,12.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243170,34,11,0
243173,0,0,16
243174,0,14,23
243185,0,0,1
243204,3,12,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243170,34,11,0
1,243173,0,0,16
2,243174,0,14,23
3,243185,0,0,1
4,243204,3,12,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243170,34,11,
1,243173,,,16
2,243174,,14,23
3,243185,,,1
4,243204,3,12,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 12:15PM,243210,20,Alumier Labs Inc.
12,Aug 2 2022 12:15PM,243210,20,"ACI Healthcare USA, Inc."
13,Aug 2 2022 12:00PM,243273,10,"Snap Medical Industries, LLC"
16,Aug 2 2022 11:33AM,243272,20,"ACI Healthcare USA, Inc."
18,Aug 2 2022 11:13AM,243268,10,Ivaoes Animal Health
23,Aug 2 2022 11:11AM,243267,15,"Mizner Bioscience, LLC"
56,Aug 2 2022 10:56AM,243266,15,"Virtus Pharmaceuticals, LLC"
75,Aug 2 2022 10:45AM,243265,10,MedStone Pharma LLC
87,Aug 2 2022 10:45AM,243253,15,"Person & Covey, Inc."
96,Aug 2 2022 10:40AM,243263,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 12:15PM,243210,20,Alumier Labs Inc.,,,13
1,Aug 2 2022 12:15PM,243210,20,"ACI Healthcare USA, Inc.",,,13
2,Aug 2 2022 12:00PM,243273,10,"Snap Medical Industries, LLC",,,3
3,Aug 2 2022 11:33AM,243272,20,"ACI Healthcare USA, Inc.",,,2
4,Aug 2 2022 11:13AM,243268,10,Ivaoes Animal Health,,,5
5,Aug 2 2022 11:11AM,243267,15,"Mizner Bioscience, LLC",,19,14
6,Aug 2 2022 10:56AM,243266,15,"Virtus Pharmaceuticals, LLC",,15,4
7,Aug 2 2022 10:45AM,243265,10,MedStone Pharma LLC,,11,1
8,Aug 2 2022 10:45AM,243253,15,"Person & Covey, Inc.",,5,4
9,Aug 2 2022 10:40AM,243263,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:15PM,243210,20,Alumier Labs Inc.,13,,
1,Aug 2 2022 12:15PM,243210,20,"ACI Healthcare USA, Inc.",13,,
2,Aug 2 2022 12:00PM,243273,10,"Snap Medical Industries, LLC",3,,
3,Aug 2 2022 11:33AM,243272,20,"ACI Healthcare USA, Inc.",2,,
4,Aug 2 2022 11:13AM,243268,10,Ivaoes Animal Health,5,,
5,Aug 2 2022 11:11AM,243267,15,"Mizner Bioscience, LLC",14,19,
6,Aug 2 2022 10:56AM,243266,15,"Virtus Pharmaceuticals, LLC",4,15,
7,Aug 2 2022 10:45AM,243265,10,MedStone Pharma LLC,1,11,
8,Aug 2 2022 10:45AM,243253,15,"Person & Covey, Inc.",4,5,
9,Aug 2 2022 10:40AM,243263,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:15PM,243210,20,Alumier Labs Inc.,13,,
1,Aug 2 2022 12:15PM,243210,20,"ACI Healthcare USA, Inc.",13,,
2,Aug 2 2022 12:00PM,243273,10,"Snap Medical Industries, LLC",3,,
3,Aug 2 2022 11:33AM,243272,20,"ACI Healthcare USA, Inc.",2,,
4,Aug 2 2022 11:13AM,243268,10,Ivaoes Animal Health,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:15PM,243210,20,Alumier Labs Inc.,13.0,NaN,NaN
1,Aug 2 2022 12:15PM,243210,20,"ACI Healthcare USA, Inc.",13.0,NaN,NaN
2,Aug 2 2022 12:00PM,243273,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
3,Aug 2 2022 11:33AM,243272,20,"ACI Healthcare USA, Inc.",2.0,NaN,NaN
4,Aug 2 2022 11:13AM,243268,10,Ivaoes Animal Health,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 12:15PM,243210,20,Alumier Labs Inc.,13.0,0.0,0.0
1,Aug 2 2022 12:15PM,243210,20,"ACI Healthcare USA, Inc.",13.0,0.0,0.0
2,Aug 2 2022 12:00PM,243273,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
3,Aug 2 2022 11:33AM,243272,20,"ACI Healthcare USA, Inc.",2.0,0.0,0.0
4,Aug 2 2022 11:13AM,243268,10,Ivaoes Animal Health,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2675635,85.0,19.0,0.0
15,1702633,63.0,53.0,0.0
16,243233,1.0,3.0,0.0
18,243232,1.0,0.0,0.0
19,2432430,13.0,14.0,3.0
20,1216118,41.0,29.0,34.0
21,729737,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2675635,85.0,19.0,0.0
1,15,1702633,63.0,53.0,0.0
2,16,243233,1.0,3.0,0.0
3,18,243232,1.0,0.0,0.0
4,19,2432430,13.0,14.0,3.0
5,20,1216118,41.0,29.0,34.0
6,21,729737,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,85.0,19.0,0.0
1,15,63.0,53.0,0.0
2,16,1.0,3.0,0.0
3,18,1.0,0.0,0.0
4,19,13.0,14.0,3.0
5,20,41.0,29.0,34.0
6,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,15,Released,63.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,3.0,34.0,0.0
Executing,19.0,53.0,3.0,0.0,14.0,29.0,1.0
Released,85.0,63.0,1.0,1.0,13.0,41.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,34.0,0.0
1,Executing,19.0,53.0,3.0,0.0,14.0,29.0,1.0
2,Released,85.0,63.0,1.0,1.0,13.0,41.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,34.0,0.0
1,Executing,19.0,53.0,3.0,0.0,14.0,29.0,1.0
2,Released,85.0,63.0,1.0,1.0,13.0,41.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()